In [20]:
import os
import cv2
import json
import shutil
import random
import numpy as np
import glob
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
from src.create_annotations import *
from shapely.geometry import Polygon, MultiPolygon
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import morphology
from skimage.filters import threshold_otsu
from skimage.transform import rotate
from skimage import exposure
from skimage.filters import unsharp_mask
from skimage.restoration import denoise_tv_chambolle
from skimage.transform import resize


In [4]:
image_list = []
image_list2 = []
for filename in glob.glob('dataset/input_images/*.jpg'):  # Assuming images are in JPG format
    im = Image.open(filename)
    image_list.append(im)
    image_list2.append(im.copy())


Skew Correction. ...
Image Scaling. ...
Noise Removal. ...
Thinning and Skeletonization. ...
Gray Scale image. ...
Thresholding or Binarization.

In [5]:
def deskew(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's thresholding
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour
    contour = max(contours, key=cv2.contourArea)

    # Get the minimum bounding rectangle of the contour
    rect = cv2.minAreaRect(contour)
    angle = rect[2]

    # Rotate the image to correct skew
    if angle < -45:
        angle += 90
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated



In [7]:
def scale_image(image, size=(200, 200)):
    return cv2.resize(image, size)

In [8]:
def remove_noise(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

In [9]:
def thin_skeletonize(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY_INV)
    skeletonized_image = morphology.skeletonize(binary_image / 255)
    return cv2.cvtColor(skeletonized_image.astype(np.uint8) * 255, cv2.COLOR_GRAY2BGR)


In [10]:
def binarize_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    threshold_value = threshold_otsu(gray_image)
    return gray_image > threshold_value


In [21]:
def sharpen_image(image):
    return unsharp_mask(image, radius=1.0, amount=1.0)

In [22]:
def enhance_contrast(image):
    p2, p98 = np.percentile(image, (2, 98))
    return exposure.rescale_intensity(image, in_range=(p2, p98))


In [23]:
def super_resolution(image):
    # Placeholder function for super-resolution, as it requires a trained model or library implementation
    return resize(image, (image.shape[0]*2, image.shape[1]*2))

In [24]:
def denoise_image(image):
    return denoise_tv_chambolle(image, weight=0.1)

In [25]:
def color_correction(image):
    # Placeholder function for color correction
    return image

In [11]:
# Create the output directory if it doesn't exist
output_dir = 'output/processed_images'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# def save_images(image_list, output_dir):

#     # Iterate over the image list
#     for idx, im in enumerate(image_list):
#         # Convert PIL image to OpenCV format
#         cv_image = np.array(im)

#         # Skew Correction
#         deskewed_image = deskew(cv_image)

#         # Image Scaling
#         scaled_image = scale_image(deskewed_image)

#         # Noise Removal
#         noise_removed_image = remove_noise(scaled_image)

#         # Thinning and Skeletonization
#         skeletonized_image = thin_skeletonize(noise_removed_image)

#         # Binarization
#         binary_image = binarize_image(skeletonized_image)

#         # Save the processed image
#         output_filename = f"{output_dir}/processed_image_{idx}.jpg"
#         cv2.imwrite(output_filename, binary_image.astype(np.uint8) * 255)

#     # Notify user that processing is complete
#     print("Image processing complete. Processed images saved in 'output/processed_images' directory.")

# # Call the function to process and save the images
# output_dir = 'output/processed_images'
# save_images(image_list, output_dir)

In [28]:
def save_processed_image(processed_image, output_dir, idx):
        output_filename = f"{output_dir}/processed_image_{idx}.jpg"
        cv2.imwrite(output_filename, processed_image.astype(np.uint8))

In [29]:
def save_images(image_list, output_dir):
# Iterate over the image list
    for idx, im in enumerate(image_list):
        # Convert PIL image to OpenCV format
        cv_image = np.array(im)

        

        # Skew Correction
        deskewed_image = deskew(cv_image)
        save_processed_image(deskewed_image,output_dir,idx)
        # output_filename = f"{output_dir}/deskewed_image_{idx}.jpg"
        # cv2.imwrite(output_filename, deskewed_image.astype(np.uint8))

        # Image Scaling
        # scaled_image = scale_image(cv_image)
        # output_filename = f"{output_dir}/scaled_image_{idx}.jpg"
        # cv2.imwrite(output_filename, scaled_image.astype(np.uint8))

        # Noise Removal
        # noise_removed_image = remove_noise(cv_image)
        # output_filename = f"{output_dir}/noise_removed_image_{idx}.jpg"
        # cv2.imwrite(output_filename, noise_removed_image.astype(np.uint8))

        # Thinning and Skeletonization
        # skeletonized_image = thin_skeletonize(cv_image)
        # output_filename = f"{output_dir}/skeletonized_image_{idx}.jpg"
        # cv2.imwrite(output_filename, skeletonized_image.astype(np.uint8))

        # Binarization
        # binary_image = binarize_image(cv_image)
        # output_filename = f"{output_dir}/binary_image_{idx}.jpg"
        # cv2.imwrite(output_filename, binary_image.astype(np.uint8))

    # Notify user that processing is complete
    print("Image processing complete. Processed images saved in 'output/processed_images' directory.")



In [31]:
# Call the function to process and save the images
output_dir = 'output/processed_images'
save_images(image_list, output_dir)

Image processing complete. Processed images saved in 'output/processed_images' directory.
